In [1]:
import numpy as np # 행렬계산
import pandas as pd # 엑셀 
import datetime # 시간인덱스를 시간포맷으로 변경
import re
from dateutil.parser import parse
import math
import glob # 파일 로드 
import sys
import openpyxl as opx # 엑셀파일 로드 (xls)
import xlrd # 엑셀파일 로드 (최신버전 xlsx)
import seaborn as sns
import random
import matplotlib # Graph
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import font_manager, rc
# matplotlib.rcParams['axes.unicode_minus'] = False
# font_path = "C:/Windows/Fonts/malgun.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
# %matplotlib inline


from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# data 추출

## 엑셀파일 concat

In [ ]:
files = glob.glob('./raw_data/power/*.xlsx')

all_data = pd.DataFrame()

for f in files:
    data = pd.read_excel(f, parse_dates=True)  # 데이터를 판다스로 리드      
    df = pd.DataFrame(data)    # 데이터프레임을 불러온 데이터로 선언
    all_data = pd.concat([all_data, df]) # 위에까지 진행된 내용에 아래를 concat 
    print(f, all_data.shape)
all_data


In [ ]:
df = all_data

In [ ]:
df = df.rename(columns={'Date': 'DateTime','16-bit Unsigned':'Power'})

## 인덱스 변경

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [ ]:
df = df.set_index('DateTime')

In [ ]:
df

## Time column drop

In [ ]:
df.drop(columns=['Time'], inplace=True)

In [ ]:
df

## CSV 파일로 export

In [ ]:
file_name = 'merged.csv'
df.to_csv(file_name)

## 1차 visualization

In [ ]:
fig, ax = plt.subplots(figsize=(18,20))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df[name])    
    plt.plot(df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


# 엑셀데이터 불러오기

In [ ]:
df = pd.read_excel('230821.xlsx', parse_dates=True)  

In [ ]:
df

# 데이터 전처리

## 마스크 설정
- 650 이상이거나, 550 이하이면 outlier로 간주

In [ ]:
mask =  ((df['Millisecond']>= 650)|(df['Millisecond'] <= 550)) 

## outlier 추출

In [ ]:
df_outlier = df[mask]

In [ ]:
df_outlier

## Filtered 추출

In [ ]:
df_filtered = df[~mask]

In [ ]:
df_filtered

## Masking된 것 만 visualization

In [ ]:
fig, ax = plt.subplots(figsize=(18,20))

for i in range(len(df_filtered.columns)):
    plt.subplot(len(df_filtered.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df_filtered.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df_filtered[name])    
    plt.plot(df_filtered[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


## Outlier visulaization

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df_outlier.columns)):
    plt.subplot(len(df_outlier.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df_outlier.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df_outlier[name])    
    plt.plot(df_outlier[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


# Resampling

## 5분단위 평균

In [ ]:
df = df_filtered

In [ ]:
df

In [ ]:
df = df[['Power']]

## 하루단위 전력량으로 환산

In [ ]:
df = df.resample('1d').sum()/12

In [ ]:
df

In [ ]:
df2

In [ ]:
df3 = df2.interpolate(method='linear')

In [ ]:
df=df2

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
#     plt.plot(df[name])    
#     plt.plot(df[name], color=color)    
    plt.bar(df.index, df[name], color=color, width=0.4)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2023-07-05 00:00:00'), pd.Timestamp('2023-07-06 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


# Power가 0인값 초과인 경우만 추출

In [ ]:
df

In [ ]:
mask =  (df['Power']> 0)

In [ ]:
df_filtered = df[mask]

In [ ]:
df=df_filtered

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df[name])    
    plt.plot(df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.scatter(df.index, df[name])    
    plt.scatter(df.index, df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


## 5분단위 평균

In [ ]:
df = df.resample('5T').mean()

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.scatter(df.index, df[name])    
    plt.scatter(df.index, df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


## CSV 파일로 저장 

In [ ]:
df.to_csv('./treated_data/230821_Power_5min_average.csv')

# PV데이터 로딩

In [2]:
files = glob.glob('./raw_data/pv/*.csv')

all_data = pd.DataFrame()

for f in files:
    data = pd.read_csv(f, header=15)  # 데이터를 판다스로 리드      
    df = pd.DataFrame(data)    # 데이터프레임을 불러온 데이터로 선언
    all_data = pd.concat([all_data, df]) # 위에까지 진행된 내용에 아래를 concat 
    print(f, all_data.shape)
all_data


./raw_data/pv\06240000.CSV (7, 29)
./raw_data/pv\06240400.CSV (32, 29)
./raw_data/pv\06290000.CSV (35, 29)
./raw_data/pv\06290100.CSV (38, 29)
./raw_data/pv\06290200.CSV (109, 29)
./raw_data/pv\06300000.CSV (399, 29)
./raw_data/pv\07010000.CSV (605, 29)
./raw_data/pv\07010100.CSV (611, 55)
./raw_data/pv\07010200.CSV (614, 55)
./raw_data/pv\07010300.CSV (619, 55)
./raw_data/pv\07010400.CSV (3073, 55)
./raw_data/pv\07030000.CSV (4771, 55)
./raw_data/pv\07030001.CSV (5771, 55)
./raw_data/pv\07120000.CSV (5974, 55)
./raw_data/pv\07120001.CSV (13166, 55)
./raw_data/pv\08070000.CSV (17840, 55)
./raw_data/pv\08070001.CSV (17847, 55)


,Date,Etime,Status,Freq_Avg[Hz],U1_Avg[V],Ufnd1_Avg[V],Udeg1_Avg[deg],I1_Avg[A],Ifnd1_Avg[A],Ideg1_Avg[deg],...,Upeak1_Min[V],Udeg1_Min[deg],I1_Min[A],Ifnd1_Min[A],Ipeak1_Min[A],Ideg1_Min[deg],P1_Min[W],S1_Min[VA],Q1_Min[var],PF1_Min
0,2023-06-24 12:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-06-24 12:45:00,0000:05:00,0.0,60.034,218.87,218.81,0.0,3.49,3.48,10.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-06-24 12:50:00,0000:10:00,0.0,60.033,218.96,218.90,0.0,3.49,3.48,10.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-06-24 12:55:00,0000:15:00,0.0,60.008,219.03,218.96,0.0,3.49,3.48,10.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-06-24 13:00:00,0000:20:00,0.0,60.003,218.54,218.49,0.0,3.49,3.49,10.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,2023-08-23 15:20:00,0389:40:00,0.0,50.108,212.80,212.60,0.0,3.55,3.33,-3.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-23 15:25:00,0389:45:00,0.0,50.119,213.62,213.44,0.0,9.37,9.35,-2.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-23 15:30:00,0389:50:00,0.0,50.153,216.91,216.74,0.0,8.14,8.06,-2.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-08-23 15:34:16,0389:54:16,0.0,50.212,215.20,215.03,0.0,8.23,8.21,-3.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
